# 1. 크롤링 단계
## 주요함수
- driver.find_element(By.XPATH, ...) html 경로 검색
- driver = webdriver.Chrome() : 셀레니움에서 웹창을 통해 정보추출
- driver.get('https://www.bomtoon.com/user/login') : 셀레니움 get방식 추출
- driver.post('https://www.bomtoon.com/user/login') : 셀레니움 get방식 추출
- while_try/except : while문에서 로그인 예외처리
- IndexError : except에서 인덱스가 없는 경우 예외처리
- with open('경로', 'a/w/r', encoding='utf-8') as f: f.write(str(dic_res)) : a(새로운 내용 추가)/w(쓰기)/r(읽기)
- pd.Series : dic 데이터 프레임화


In [1]:
#https://www.bomtoon.com/bom/comic/weekly


# 셀레늄 설치 여부
!pip list | findstr selenium

# 설치 명령
!pip install selenium
!pip install pyperclip

#https://chromedriver.chromium.org/downloads

#Chrome 업데이트 중 https://chromedriver.com/download#stable
#버전 123.0.6312.58(공식 빌드) (64비트)  https://storage.googleapis.com/chrome-for-testing-public/123.0.6312.58/win64/chromedriver-win64.zip

/bin/bash: line 1: findstr: command not found
ERROR: Pipe to stdout was broken
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='utf-8'>
BrokenPipeError: [Errno 32] Broken pipe
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00


In [2]:
import os
os.getcwd()

'/content'

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import pyperclip
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os


driver = webdriver.Chrome()
driver.get('https://www.bomtoon.com/user/login')
nid='om'
npw='()'

time.sleep(3)
driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div/div/div[4]/input[1]').send_keys(nid)
time.sleep(3)
driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div/div/div[4]/input[2]').send_keys(npw)
time.sleep(3)
driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div/div/div[6]').click()

driver.implicitly_wait(10)

time.sleep(3)
co=driver.get_cookies()
print(type(co))

for i in co:
    print(i)

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/123.0.6312.122/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x592de4bdd863 <unknown>
#1 0x592de48d38c6 <unknown>
#2 0x592de4907d34 <unknown>
#3 0x592de4903d3d <unknown>
#4 0x592de494caed <unknown>
#5 0x592de4940343 <unknown>
#6 0x592de4911593 <unknown>
#7 0x592de4911f5e <unknown>
#8 0x592de4ba184b <unknown>
#9 0x592de4ba57a5 <unknown>
#10 0x592de4b8f571 <unknown>
#11 0x592de4ba6332 <unknown>
#12 0x592de4b7487f <unknown>
#13 0x592de4bcc728 <unknown>
#14 0x592de4bcc8fb <unknown>
#15 0x592de4bdc9b4 <unknown>
#16 0x7f71c8429ac3 <unknown>


In [ ]:
%config Completer.use_jedi = False #jupyter note 탭기능이 안될때 사용

In [ ]:
!pip install pyperclip # ctrl + C, ctrl + V 기능을 가진 라이브러리

In [ ]:
import pandas as pd
import time
import pyperclip
import os
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime,timedelta
from tqdm import tqdm


global titleUrl

titleUrl='https://www.bomtoon.com/api'

url = titleUrl+'/balcony-api-v2/contents/tab/details'


headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
  'X-Balcony-Id': 'BOMTOON_COM',
  'Content-Type': 'application/json',
  'Authorization': 'Bearer testw'}


#data ={'contentsIds':'34157,33735,20381,14320,20617,34064,18147,16531,33753,34138,29926,22965,32089,21329,27572,28753,32598,30044,22699,21149,34077,28480,26487','contentsThumbnailType':'VERTICAL,MAIN,SQUARE'}

def trans_datatime(unix_timestamp):
    unix_timestamp_sec = unix_timestamp / 1000
    dt_object = datetime.utcfromtimestamp(unix_timestamp_sec)
    korea_dt_object = dt_object + timedelta(hours=9)
    formatted_date = korea_dt_object.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_date

def AweekUrl(headers_):   #주간 작가의 id 전체 data 추출 함수
    aweekAll_list=[]
    aweek=['monday','tuesday','wednesday','thursday','friday','saturday','sunday','ten','COMPLETE']
    for i in aweek:
        valWeek=titleUrl+'/balcony-api-v2/contents/tab/schedule/comic?groupMenu={}&isIncludeTen=false&sort=POPULAR&adultToggle=false'.format(i.upper())
        res = requests.request("GET", valWeek, headers=headers_)
        #print(valWeek)
        aweekJson = res.json()
        #print(aweekJson['data'])
        aweekAll_list.append(aweekJson['data'])

    return aweekAll_list

#https://www.bomtoon.com/api/balcony-api-v2/contents/tab/details

#https://www.bomtoon.com/comment/작가

def GetBomJakas(url_,headers_,getData): # 작가당 웹툰 작가 이름 및 ID 2차원 리스트

    jakaList_name=[[]]

    res=requests.post(url_,  headers=headers_, json=getData)
    dic_res=res.json()
    for i in range(len(dic_res['data'])):
        #print(dic_res['data'][i]['alias'])
        jakaList_name.append([dic_res['data'][i]['alias'],dic_res['data'][i]['creators']])

    return jakaList_name # 작가당 웹툰 작가 이름 및 ID 2차원 리스트


def getCustData(createData):  # data 데이터 재생성 반화
    val=','.join(map(str,createData))
    getDic={'contentsIds' : val ,'contentsThumbnailType':'VERTICAL,MAIN,SQUARE'}
    return getDic


def getPage_count(getID,headers_):  # 개별 작가의 댓글 총갯수
    totElement=0
    try:
        url_=titleUrl+'/balcony-api/comment/contents/{}{}'.format(getID,'?isBest=true&page=0')
        res = requests.request("GET", url_, headers=headers_)
        dic_res=res.json()
        if dic_res['data'] == None:
            print('no data')
        else:
            totElement=int(dic_res['data']['comment']['totalPages'])

    except IndexError: ##인덱스 벗어나는 에러를 잡았을 경우에 대해서만!
        print("페이지가 없습니다.")

    return totElement  # 개별 작가의 댓글 총갯수



#작가명,작가ID,댓글총갯수,댓글총페이지 | 게시물인덱스,댓글쓴날짜,댓글의idx,작품명,작품회차,댓글자명,댓글,좋아요횟수
c_write,c_writerID,c_replayCount,c_replayPage,c_commtIdx,c_day,c_nanoreIdx,c_title,c_cha,c_replayName,c_comment,c_like=[],[],[],[],[],[],[],[],[],[],[],[]

selfcwd=os.getcwd()
fold=selfcwd.split('\\')


def GetAll_Comment(getID, getName, headers_, pageCount=0):
    # print(titleUrl+'/balcony-api/comment/contents/{}'.format(getID[0]))
    start = time.time()
    totElement = 0
    print('==>! ',getID, getName, pageCount)
    try:
        for su in range(int(pageCount)):
            url_ = titleUrl + '/balcony-api/comment/contents/{}?isBest=true&page={}'.format(getID, su)
            print('_COMMENT :==> ', url_)
            res = requests.request("GET", url_, headers=headers_)
            time.sleep(2)
            dic_res = res.json()
            #print('res 갯수 : ',len(dic_res))

            # 추가 모드로 파일 열기
            with open('C:\\Users\\'+fold[2]+'\\Downloads\\commentJSON2.txt', 'a', encoding='utf-8') as f:
                f.write(str(dic_res))


            if dic_res['data'] == None:
                    print('no data')
                    c_comment.append('page no data')
            else:
                totElement = int(dic_res['data']['comment']['numberOfElements'])

                #print(f'작가명 : {getName}__ {getID}')

                loopdata = dic_res['data']['comment']['content']
                jIDX = 0
                try:
                    for j in range(25):   # 0페이지 댓글 전체만 추출 추출
                            # 여기중복
                            #print(f'작가명 : {getName[0]}')
                            c_write.append(getName)
                            #print(f'작가명ID : {getID[0]}')
                            c_writerID.append(getID)
                            #print('댓글 총 수 : {}'.format(totElement))
                            c_replayCount.append(totElement)
                            #print('댓글 총 페이지 수 : {}'.format(pageCount))
                            c_replayPage.append(pageCount)
                            # 여기중복 끝

                            ymd = trans_datatime(loopdata[j]['createdAt'])

                            #print(f'게시물 IDX :: {jIDX}')
                            c_commtIdx.append(jIDX)
                            jIDX += 1
                            # print(ymd)
                            c_day.append(ymd)

                           # print(loopdata[j]['commentId'], loopdata[j]['title'],
                           #     loopdata[j]['subTitle'], loopdata[j]['nickname'])

                            c_nanoreIdx.append(loopdata[j]['commentId'])
                            c_title.append(loopdata[j]['title'])
                            c_cha.append(loopdata[j]['subTitle'])
                            c_replayName.append(loopdata[j]['nickname'])
                        #print(loopdata[j]['comment'],loopdata[j]['likeCount'] )
                            c_comment.append(loopdata[j]['comment'])
                            #print(loopdata[j]['comment'])
                            c_like.append(loopdata[j]['likeCount'])

                except IndexError:
                    print('page end')
                    dic_res={}

    except IndexError:  # 인덱스 벗어나는 에러를 잡았을 경우에 대해서만!
        print("페이지가 없습니다.")


    end = time.time()

    print(f"{(end - start)*60/10:.5f} sec")
    return 0


writerLists=AweekUrl(headers)  #주간 작가의 id 전체 data 추출 함수

realdata=getCustData(writerLists[8]) # post 전송데이터 생성
twoList=GetBomJakas(url,headers,realdata)  #  [ [getID],[getName]]

getID=list(map(lambda x: x[0], twoList[1:]))
getName=list(map(lambda x: x[1], twoList[1:]))
print(len(getID) , len(getName))
    #print('tqdm ==> ' ,getID[t] )
for p in tqdm(range(len(getID))):
    pageCount=getPage_count(getID[p],headers)
    #print(f'pageChk ==> {pageCount} :: {getID[p]}')
    GetAll_Comment(getID[p],getName[p],headers,pageCount)

In [ ]:
import pandas as pd
pddata = {'작가명': pd.Series(c_write)
          ,'작가ID': pd.Series(c_writerID)
          ,'댓글총갯수':pd.Series(c_replayCount)
          ,'댓글총페이지':pd.Series(c_replayPage)
          ,'게시물인덱스':pd.Series(c_commtIdx)
          ,'댓글쓴날짜':pd.Series(c_day)
          ,'댓글의idx':pd.Series(c_nanoreIdx)
          ,'작품명':pd.Series(c_title)
          ,'작품회차':pd.Series(c_cha)
          ,'댓글자명':pd.Series(c_replayName)
          ,'댓글':pd.Series(c_comment)
          ,'좋아요횟수':pd.Series(c_like)

       }
df = pd.DataFrame(pddata,columns=['작가명','작가ID','댓글총갯수','댓글총페이지','게시물인덱스','댓글쓴날짜','댓글의idx','작품명','작품회차','댓글자명','댓글','좋아요횟수'])

#작가명,작가ID,댓글총갯수,댓글총페이지 | 게시물인덱스,댓글쓴날짜,댓글의idx,작품명,작품회차,댓글자명,댓글,좋아요횟수
#del[[df]]

#df
selfcwd=os.getcwd()
fold=selfcwd.split('\\')
# 데이터프레임을 CSV 파일로 저장

## replay titleUrl+'/balcony-api/comment/reply/CONTENTS/1654517?isBest=true&page=0
if not os.path.exists('C:\\Users\\'+fold[2]+'\\Downloads\\output.csv'):
    df.to_csv('C:\\Users\\'+fold[2]+'\\Downloads\\output.csv', index=False, mode='w', encoding='utf-8-sig')
else:
    df.to_csv('C:\\Users\\'+fold[2]+'\\Downloads\\output.csv', index=False, mode='a', encoding='utf-8-sig', header=False)


# 2. 크롤링 데이터 분석
## 주요함수
- pd.read_csv('경로.파일형식') : 데이터 읽기
- df.info : 데이터 프레임 정보표시
- df.isna().sum() : 결측츠 표시
- df.dropna(변수 = ['열']) : 결측치 있는 특정 열의 행 제거
- df.to_csv('파일명.파일형식', index = False) : 파일 저장, 인덱스 없음


In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/bomAll_nan포함.csv')
df1

In [ ]:
df1.info() # 데이터 프레임 정보 표시

In [ ]:
df1.isna().sum() # 결측치 표시

In [ ]:
df1 = df1.dropna(subset = ['댓글']) # 결측치 있는 댓글 행 제거

In [ ]:
df1.to_csv("AllReviewMVDrop_BUM.csv", index = False)

# 3. 리뷰 코퍼스 작성
## 주요함수
- 변수.head() : 불러온 데이터 요약 확인, 기본 5줄
- [review for review in review_data['댓글'] if type(review) is str] : for 문, review data의 타입이 str이면 불러들임
- pd.DataFrame(key : value) : key, value 데이터 프레임화


In [ ]:
import pandas as pd

review_data = pd.read_csv('/content/boom_final.csv') #결측치 제거된 데이터 업로드 후 읽음 "봄툰csv 디렉터리의 dr"
review_data.head() #불러온 데이터 빠르게 확인

print('데이터 개수: {}'.format(len(review_data)))

# 문자열 아닌 데이터 모두 제거
review_data = [review for review in review_data['댓글'] if type(review) is str]

# 리스트를 DataFrame으로 변환
review_df = pd.DataFrame({'댓글': review_data})

# CSV 파일로 내보내기
review_df.to_csv('AllReviewCorpus.csv', index=False)

# XLSX 파일로 내보내기 (openpyxl 라이브러리가 필요할 수 있습니다)
review_df.to_excel('AllReviewCorpus.xlsx', index=False)

# 4. 전처리
## 주요함수
- from konlpy.tag import Okt : Okt 형태소 라이브러리, 태그 포함
- np.array(변수) : numpy
- re.sub(r'[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-zA-Z0-9]+', '', review) : review 특수문자, 숫자, 한글 모음/자음 공란 변경하는 정규화 표현식
- okt.nouns(변수) : 형태소 분석 중 명사 추출
- ' '.join([noun for noun in nouns if noun not in stopwords]) : 복수의 공란 한 개로 변경

In [ ]:
# !pip install konlpy 인스톨하세요

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 데이터 불러오기
data = pd.read_csv('/content/AllReviewCorpus_Moon.csv')
print('데이터 개수:', len(data))

# 문자열이 아닌 데이터 제거
data_1 = data['댓글'].dropna().map(str)

# 불용어 목록 파일을 불러옵니다.
with open('/content/stopword_ko_Main.txt', 'r', encoding='utf-8') as f:
    stopwords = np.array([line.strip().replace('\r', '') for line in f.readlines()])

# 명사 추출 및 불용어 처리
filtered_review = []
for review in tqdm(data_1, desc="리뷰 처리 중"):
    review = re.sub(r'[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-zA-Z0-9]+', '', review)
    nouns = okt.nouns(review)  # 형태소 분석을 통해 명사만 추출

    # numpy 배열을 사용하여 불용어를 필터링합니다. 이 경우, 리스트 컴프리헨션을 사용하는 것이 더 적합합니다.
    filtered_nouns = ' '.join([noun for noun in nouns if noun not in stopwords])
    filtered_review.append(filtered_nouns)

# 4. 전처리
## 주요함수
- fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' : 나눔바른고딕 import
- WordCloud(font_path=폰트 변수, background_color='색', width=가로 크기, height=세로 크기, stopwords=set(불용어 변수, 중복제거)).generate(' '.join(filtered_review))
- plt.figure(figsize=(가로, 세로))
- plt.imshow(워드클라우드 변수, interpolation='표시 형식')
- plt.axis('off') :
- plt.show() : 워드클라우드 표시

In [ ]:
# !apt -qq -y install fonts-nanum > /dev/null #한글 글꼴 설치
# !pip install wordcloud

import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt

data = pd.read_csv('/content/filtered_review.csv') #워드클라우드 파일 불러오기

# 다시 numpy 배열로 변환합니다.
filtered_review = np.array(filtered_review)

# 워드 클라우드 생성 및 출력
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
wordcloud = WordCloud(font_path=fontpath, background_color='white', width=800, height=800, stopwords=set(stopwords)).generate(' '.join(filtered_review))
plt.figure(figsize=(8, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

# 5. PolyLDA
## 주요함수
- TfidfVectorizer(max_features= 최대 열, max_df = 0.5, smooth_idf=True)
- vectorizer.fit_transform(변수)
- LatentDirichletAllocation(n_components=20, random_state=777)
- lda_model.fit(X)
- def get_topics(components, feature_names, n=5):
     for idx, topic in enumerate(components):
         print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
-pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(
                        lda_model = lda_model,
                       dtm = X,
                       vectorizer = vectorizer,
                       mds='tsne')
- pyLDAvis.display(변수)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups

In [ ]:
vectorizer = TfidfVectorizer(max_features= 1000, # 상위 1,000개의 단어를 보존
max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(clean_review[0])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
# 모델 객체 생성
lda_model = LatentDirichletAllocation(n_components=20, random_state=777)

In [ ]:
# fit transfom
lda_model.fit(X)

In [ ]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names_out()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

In [ ]:
# 시각화
# LDAvis 는 토픽 모델링에 자주 이용되는 Latent Dirichlet Allocation (LDA) 모델의 학습 결과를 시각적으로 표현하는 라이브러리입니다
!pip install -U pyLDAvis

In [ ]:
# LDA 시각화를 진행해봅시다.
import pyLDAvis
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(
                        lda_model = lda_model,
                       dtm = X,
                       vectorizer = vectorizer,
                       mds='tsne')
pyLDAvis.display(vis)